In [1]:
import pandas as pd
df = pd.read_csv('https://storage.googleapis.com/jsingh-bigdata-public/chicago-taxi-rides.csv')
df.dropna(inplace = True)
df.reset_index(inplace=True, drop=True)
df = df[df.columns].astype(int)
df

,pickup_community_area,dropoff_community_area,trips
0,14,19,2107
1,14,24,5370
2,29,29,2759
3,29,25,313
4,41,44,1693
...,...,...,...
5395,14,55,1
5396,10,65,3
5397,49,59,2
5398,20,74,1


In [2]:
from scipy import sparse
import numpy as np
pd.options.display.float_format = '{:f}'.format
np.set_printoptions(suppress=True)
matrix = sparse.coo_matrix((df.trips.values, (df.pickup_community_area-1, df.dropoff_community_area-1)), (77, 77))
matrix = matrix.toarray()
matrix = matrix.astype('float')
col_sums = np.sum(matrix, axis = 0)
for i in range(77): # normalize it by the total distance summed within columns
    matrix[:,i] = matrix[:,i]/float(col_sums[i])

In [3]:
init = np.ones((77, 77))/77.0
beta = 0.85

ranks_old = np.ones((77, 1))/77.0
power = [2**i for i in range(1, 7)]
result_rank = list()

# matrix A
A = (beta * matrix) + ((1 - beta)*init)

for i in range(64):
    ranks_old = np.matmul(A,ranks_old)
    if i+1 in power:
        result_rank.append(ranks_old.reshape(77,))

In [4]:
reshape_result = np.array(result_rank).T
column_names = ["rank " + str(i) for i in power]
result_df = pd.DataFrame(reshape_result, columns = column_names)
result_df

,rank 2,rank 4,rank 8,rank 16,rank 32,rank 64
0,0.006221,0.005239,0.005016,0.005009,0.005009,0.005009
1,0.007271,0.005592,0.005217,0.005204,0.005204,0.005204
2,0.016433,0.015223,0.014841,0.014829,0.014829,0.014829
3,0.007397,0.006154,0.005911,0.005905,0.005905,0.005905
4,0.007654,0.007047,0.006930,0.006928,0.006928,0.006928
...,...,...,...,...,...,...
72,0.002987,0.002549,0.002493,0.002492,0.002492,0.002492
73,0.002104,0.002065,0.002064,0.002064,0.002064,0.002064
74,0.002511,0.002291,0.002274,0.002273,0.002273,0.002273
75,0.047121,0.046942,0.047008,0.047017,0.047017,0.047017


In [5]:
result_df.sum(axis=0)

rank 2    1.000000
rank 4    1.000000
rank 8    1.000000
rank 16   1.000000
rank 32   1.000000
rank 64   1.000000
dtype: float64

In [17]:
# Indices of N largest elements in list
# using sorted() + lambda + list slicing
rank_64 = result_df['rank 64'].argsort()
print(rank_64[:10] + 1)
print(rank_64[-10:]+ 1)

0     9
1    74
2    72
3    52
4    18
5    20
6    26
7    75
8    29
9    47
Name: rank 64, dtype: int64
67     3
68    56
69    33
70    24
71     7
72    76
73     6
74    28
75    32
76     8
Name: rank 64, dtype: int64
